In [1]:
import cv2
import numpy as np
import os

In [2]:
face_path = "face model/"
age_path = "age model/"
in_image_path = "images/"
confidence_score = 0.6
age_buckets = ["(0-2)", "(4-6)", "(8-12)", 
               "(15-20)", "(25-32)","(38-43)", 
               "(48-53)", "(60-100)"]

In [3]:
protoFile = os.path.join(face_path, "face.prototxt")
print(protoFile)

face model/face.prototxt


In [4]:
print("[INFO] Loading the model - face detector...\n")
face_proto_path = os.path.join(face_path, "face.prototxt")
face_weights_path = os.path.join(face_path, "res10_300x300_ssd_iter_140000.caffemodel")

# reading the models and loading the weights to the model
face_model = cv2.dnn.readNet(face_proto_path, face_weights_path)
print("[INFO] Loaded the face model...")

[INFO] Loading the model - face detector...

[INFO] Loaded the face model...


In [5]:
print("[INFO] Loading the age detector model..\n")
age_proto_path = os.path.join(age_path, "age.prototxt")
age_weights_path = os.path.join(age_path, "age_net.caffemodel")

# reading the weights and making the model.

age_model = cv2.dnn.readNet(age_proto_path, age_weights_path)
print("[INFO] Loaded the age model..")

[INFO] Loading the age detector model..

[INFO] Loaded the age model..


In [6]:
image = "shehzen.png"
in_image = cv2.imread(os.path.join(in_image_path, image))

(h, w) = in_image.shape[:2]
print(f"[INFO] The shape of image is: {in_image.shape}")
image_blob = cv2.dnn.blobFromImage(in_image, 1.0, 
                                   (300,300), 
                                   (104.0, 177.0, 123.0))
# blobFromImage(image, scale, resize, mean_subtraction)
print("[INFO] Computing Face detections...")
face_model.setInput(image_blob)
detections = face_model.forward()
print("[INFO] completed!..")

[INFO] The shape of image is: (707, 567, 3)
[INFO] Computing Face detections...
[INFO] completed!..


In [7]:
detections # list of all the detections, which it thinks is a face

array([[[[0.        , 1.        , 0.9996507 , ..., 0.3235973 ,
          0.68651354, 0.7475575 ],
         [0.        , 1.        , 0.12607715, ..., 0.17958589,
          0.5911449 , 0.20981   ],
         [0.        , 1.        , 0.11718853, ..., 4.011892  ,
          4.8434434 , 4.9840035 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [8]:
detections.shape

(1, 1, 200, 7)

In [9]:
detections[0,0,0,2]

0.9996507

In [10]:
for i in range(0, detections.shape[2]):
    confidence = detections[0,0,i, 2] 
    
    if confidence > confidence_score:
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        (x1, y1, x2, y2) = box.astype("int")
        face = in_image[y1:y2, x1:x2]
        # cv2.imshow("image", face)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        face_blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), 
                                         (78.42, 87.76, 114.89), swapRB=False)
        age_model.setInput(face_blob)
        preds = age_model.forward()
        # print(preds)
        i = preds[0].argmax()
        age = age_buckets[i]
        
        age_conf = preds[0][i]
        text = "{}: {:.2f}%".format(age, age_conf*100)
        
        y = y1 - 10 if y1 - 10 > 10 else y +10
        
        cv2.rectangle(in_image, (x1, y1), (x2, y2), (0, 0, 255), 5)
        
        cv2.putText(in_image, text, (x1, y),cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0,255,255), 2)
cv2.imshow("Age: ", in_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
detections[0,0,0,3:7]

array([0.332358  , 0.3235973 , 0.68651354, 0.7475575 ], dtype=float32)